Students should write a more consise code.
The detailed coding here is for demonstration.

In [ ]:
#SVM and LR 1: Tf-idf with features 2: Tf-idf without features 3: Aravec with features 4: Aravec without features

In [ ]:
#pip install sklearn-pandas

In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn_pandas import DataFrameMapper
from sklearn.svm import LinearSVC

In [2]:
from google.colab import files

uploaded = files.upload()

Saving dataall.xlsx to dataall.xlsx


In [3]:
# use only the first columns
df = pd.read_excel("dataall.xlsx", usecols=lambda col: col not in ["id","Unnamed: 0","Unnamed: 0.1","Unnamed: 0.2"])
#or df.drop([["id","Unnamed: 0","Unnamed: 0.1","Unnamed: 0.2"]], axis=1)

In [4]:
df.head(5)

,class,tweet,hashtag,url,number
0,not,سنين نعاني وبعد ما شفنا النعمه مع الاتصالات ال...,1,0,0
1,spam,#فايبر_stc_يفرق صاحب هذه الفاتوره تنازل عن سته...,1,1,0
2,not,الف ميغا سرعه الانترنت شهريا في بريطانيا بسعر ...,1,0,0
3,not,وهذا سعرها الانترنت في بريطانيا والسرعه بالغيغ...,1,1,0
4,spam,#طيف_القحطاني #الراتب #فايبر_stc_يفرق حسبي اله...,3,0,0


In [5]:
df = df.drop_duplicates()


In [6]:
#basic descriptive stats (project note)
df.describe()

,hashtag,url,number
count,6786.000000,6786.000000,6786.000000
mean,1.739464,0.199234,0.033746
std,2.486852,0.399454,0.180588
min,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000
75%,2.000000,0.000000,0.000000
max,25.000000,1.000000,1.000000


In [7]:
df.groupby('class').describe().T

class                  not         spam
hashtag count  4270.000000  2516.000000
        mean      0.963700     3.056041
        std       0.934913     3.527743
        min       0.000000     0.000000
        25%       0.000000     1.000000
        50%       1.000000     1.000000
        75%       1.000000     4.000000
        max      13.000000    25.000000
url     count  4270.000000  2516.000000
        mean      0.118970     0.335453
        std       0.323791     0.472242
        min       0.000000     0.000000
        25%       0.000000     0.000000
        50%       0.000000     0.000000
        75%       0.000000     1.000000
        max       1.000000     1.000000
number  count  4270.000000  2516.000000
        mean      0.008197     0.077107
        std       0.090174     0.266813
        min       0.000000     0.000000
        25%       0.000000     0.000000
        50%       0.000000     0.000000
        75%       0.000000     0.000000
        max       1.000000     1.000000

In [8]:
y = df["class"]
y.value_counts()

not     4270
spam    2516
Name: class, dtype: int64

In [9]:
#we need to encode the class labels in the target variable as numbers to ensure compatibility with some models in Scikit-learn. Because we have binary classes, let's use LabelEncoder and set 'spam' = 1 and 'not' = 0.
le = preprocessing.LabelEncoder()
y_enc = le.fit_transform(y)
#print(y_enc)

In [10]:
vect = TfidfVectorizer()

X_train,X_test,Y_train,Y_test = train_test_split(df["tweet"],y_enc, test_size = 0.2, random_state = 10) #80/20 training and test set split. random state Controls the shuffling applied to the data before applying the split. 
vect.fit(X_train)
X_train_df = vect.transform(X_train)
X_test_df = vect.transform(X_test)
type(X_train_df)

scipy.sparse.csr.csr_matrix

# **Spam Detection Using Logistic Regression and Tf-idf**

In [11]:
#Machine Learning (classification) LogisticRegression
LRmodel = LogisticRegression()
LRmodel.fit(X_train_df,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
# Making predictions

prediction = dict()

prediction["Logistic"] = LRmodel.predict(X_test_df)
print(prediction["Logistic"])

[0 1 1 ... 1 1 0]


In [13]:
# Reviewing the metrics
print ("Accuracy:",accuracy_score(Y_test,prediction["Logistic"]))

Accuracy: 0.9094256259204713


In [14]:
print(classification_report(Y_test,prediction["Logistic"]))
#macro average (Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.), 
#weighted average (Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). 
#This alters ‘macro’ to account for label imbalance)

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       832
           1       0.93      0.83      0.88       526

    accuracy                           0.91      1358
   macro avg       0.91      0.89      0.90      1358
weighted avg       0.91      0.91      0.91      1358



# **Spam Detection Using Logistic Regression and Tf-idf + Other Features**

In [17]:
#Add the features of the dataframe that you want to transform and/or combine
mapper = DataFrameMapper([
     ("tweet", TfidfVectorizer()),
     ("hashtag", None),
     ("url", None),
     ("number", None)
 ])

In [18]:
"""
Use the fit_transform method to transform the old dataframe into a new one
that can be fed to the machine learning algorithm.
"""
features = mapper.fit_transform(df)
 
# Split the data between train and test
x, x_test, y, y_test = train_test_split(features,y_enc,test_size=0.2,train_size=0.8, random_state = 10)
 
LRmodel2 = LogisticRegression()
LRmodel2.fit(x,y)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
prediction = dict()

prediction["Logistic2"] = LRmodel2.predict(x_test)
print(prediction["Logistic2"])
print ("Accuracy:",accuracy_score(y_test,prediction["Logistic2"]))
print(classification_report(y_test,prediction["Logistic2"]))

[0 1 1 ... 1 1 0]
Accuracy: 0.8836524300441826
              precision    recall  f1-score   support

           0       0.86      0.96      0.91       832
           1       0.93      0.76      0.84       526

    accuracy                           0.88      1358
   macro avg       0.89      0.86      0.87      1358
weighted avg       0.89      0.88      0.88      1358



# **Spam Detection Using Support Vector Machine (SVM) and Tf-idf**

In [20]:
svm = LinearSVC().fit(X_train_df,Y_train)

In [21]:
prediction["SVM"] = svm.predict(X_test_df)
print(prediction["SVM"])

[0 1 1 ... 1 1 0]


In [22]:
accuracy_score(Y_test,prediction["SVM"])

0.9256259204712813

In [23]:
print(classification_report(Y_test,prediction["SVM"]))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94       832
           1       0.93      0.87      0.90       526

    accuracy                           0.93      1358
   macro avg       0.93      0.92      0.92      1358
weighted avg       0.93      0.93      0.93      1358



# **Spam Detection Using Support Vector Machine (SVM) and Tf-idf + Other Features**

In [24]:
svm2 = LinearSVC().fit(x,y)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [25]:
prediction["SVM2"] = svm2.predict(x_test)
print(prediction["SVM2"])
print ("Accuracy:",accuracy_score(y_test,prediction["SVM2"]))
print(classification_report(y_test,prediction["SVM2"]))

[0 1 1 ... 1 1 0]
Accuracy: 0.9167893961708394
              precision    recall  f1-score   support

           0       0.91      0.96      0.93       832
           1       0.94      0.84      0.89       526

    accuracy                           0.92      1358
   macro avg       0.92      0.90      0.91      1358
weighted avg       0.92      0.92      0.92      1358



# **Naive Bayes (just for demo)**

In [27]:
#Machine Learning (classification) Naive Bayes
from sklearn.naive_bayes import MultinomialNB
NBmodel = MultinomialNB().fit(X_train_df,Y_train)

In [28]:
prediction["Naive"] = NBmodel.predict(X_test_df)
print(prediction["Naive"])


[0 1 0 ... 1 1 0]


In [29]:
accuracy_score(y_test,prediction["Naive"])

0.8564064801178203

In [30]:
print(classification_report(y_test,prediction["Naive"]))

              precision    recall  f1-score   support

           0       0.81      0.99      0.89       832
           1       0.98      0.64      0.78       526

    accuracy                           0.86      1358
   macro avg       0.90      0.82      0.84      1358
weighted avg       0.88      0.86      0.85      1358



In [31]:
pd.DataFrame(
    confusion_matrix(y_test, prediction["Naive"]),
    index=[['actual', 'actual'], ['not', 'spam']],
    columns=[['predicted', 'predicted'], ['not', 'spam']]
)

predicted     
                  not spam
actual not        826    6
       spam       189  337

In [32]:
#print('predicted:', NBmodel.predict(X_test_df)[0])
#print('expected :', df["class"][0])

In [33]:
#Train the classifier on the whole dataset and take a look at the top 20 n-grams that are most predictive of spam.
NBmodelfull = MultinomialNB().fit(vect.fit_transform(df["tweet"]),y_enc)
pd.Series(
    NBmodelfull.coef_.T.ravel(),
    index=vect.get_feature_names()
).sort_values(ascending=False)[:20]

حي_الصيف_مع_stc   -5.932152
com               -6.074962
twiter            -6.250893
pic               -6.274749
اله               -6.281557
فايبر_stc_يفرق    -6.386815
من                -6.488232
في                -6.503429
علي               -6.700200
خصم_stc_للاجهزه   -6.778459
https             -7.024761
لا                -7.066007
المضافه           -7.198525
الي               -7.269837
عاده_صعب_تتركها   -7.272390
ان                -7.334809
خصم               -7.371422
الهم              -7.396273
stc5g             -7.400850
البنزين           -7.460378
dtype: float64

In [34]:
#a function that'll decide whether a string is spam or not, and apply it on a hypothetical message 
def spam_filter(message):
    if NBmodelfull.predict(vect.transform([message])):
        return 'spam'
    else:
        return 'not spam'

In [35]:
print(spam_filter('حلو مره'))
print(spam_filter('زين ممتاز'))

not spam
not spam
